In [1]:
import os
import numpy as np
import matplotlib as plt
import cv2

In [2]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode= False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

In [4]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for res in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                    image,
                    res,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

In [7]:
data_dir = 'Data Directory'
action_list = [
    'A',
    'BA',
    'KA',
    'DA',
    'E',
    'GA',
    'HA',
    'I',
    'LA',
    'MA',
    'NA',
    'NGA',
    'O',
    'PA',
    'RA',
    'SA',
    'TA',
    'U',
    'WA',
    'YA',
    'None'
]

In [8]:
label_map = {label:num for num, label in enumerate(action_list)}

In [ ]:
for action in os.listdir(data_dir):
    print(f'{action}')
    
    labels, sequences = [],[]
        
    for frame in os.listdir(os.path.join(data_dir, action)):
        img_path =  os.path.join(data_dir, action, frame)
            
        if os.path.exists(img_path):             
            cv2.waitKey(1)
                
            frame = cv2.imread(img_path)
                
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = hands.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
                
            draw_landmarks(image = image, results=results)
                
            if results.multi_handedness is not None:
                data = []
                for res in results.multi_hand_landmarks:
                    data = [[rex.x, rex.y, rex.z] for rex in res.landmark]
                    
                print(np.array(data).shape)
                    
                if np.array(data).shape == (21,3):
                    sequences.append(data)
                    labels.append(label_map[action])
                    print(np.array(sequences).shape)
                else:
                    print(f'ERROR: {img_path}')
                        
            draw_landmarks(image = image, results=results)
            cv2.imshow('FEED', image)
            
    #try to save evertthing
    try:
        print(f'Saving {action} data: {np.array(sequences).shape}')
        save_path = os.path.join(data_dir, action)
        np.save(os.path.join(save_path, 'labels.npy'), labels)
        np.save(os.path.join(save_path, 'sequences.npy'), sequences)
    except Exception as e:
        print(e)
            
cv2.destroyAllWindows()

In [ ]:
sequences, labels = [],[]

for action in os.listdir(data_dir):
        
    load_path = os.path.join(data_dir, action)
    label = np.load(os.path.join(load_path, 'labels.npy'))
    sequence = np.load(os.path.join(load_path, 'sequences.npy'))
        
    print(f'{load_path}')
    print(sequence.shape)
        
    sequences.append(sequence)
    labels.append(label)
        
cv2.destroyAllWindows()

np.save(os.path.join(data_dir, 'sequences.npy'), np.concatenate(sequences))
np.save(os.path.join(data_dir, 'labels.npy'), np.concatenate(labels))